In [1]:
%cd /Users/gardenmini/PycharmProjects/get_dataset/health

/Users/gardenmini/PycharmProjects/get_dataset/health


In [2]:
import pandas as pd
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
health = pd.read_csv('health.txt', index_col=0)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
data = health.drop(columns=['YEAR_t', 'trainset']).dropna()

In [5]:
data.index = np.arange(0,147472)

In [6]:
data.to_pickle('health.pkl')

In [7]:
with open('health.pkl', 'rb') as f:
    df = pickle.load(f)

In [8]:
df.ClaimsTruncated = df.ClaimsTruncated.astype('int').astype('category')

In [9]:
age = df.apply(lambda x: 0 if x.loc['age_05':'age_65'].sum()==1 else 1, axis=1)

In [10]:
df['age'] = age

In [11]:
df = df.drop(columns=['age_05', 'age_15', 'age_25', 'age_35'
                 , 'age_45', 'age_55', 'age_65', 'age_75'
                 , 'age_85'])

In [12]:
df.DaysInHospital = df.DaysInHospital.apply(lambda x: 0 if x == 0 else 1)

In [13]:
categorical = ['ClaimsTruncated', 'DaysInHospital', 'age_MISS', 'sexMALE', 'sexFEMALE',
               'sexMISS', 'labNull', 'drugNull']

In [14]:
df_categorical = df[categorical]
df_numerical = df.drop(columns = categorical)

In [15]:
hot_categorical = pd.concat([df_categorical[x].astype('int').astype('category') for x in df_categorical], axis=1)
hot_categorical.columns = df_categorical.columns

In [16]:
one_hot_numeric = pd.concat([pd.cut(df_numerical[name], 2, duplicates='drop').astype('category').cat.codes for name in df_numerical], axis=1)
one_hot_numeric.columns = df_numerical.columns

In [17]:
one_hot = pd.concat([hot_categorical.iloc[:, 2:], one_hot_numeric, hot_categorical.iloc[:, 0:2]], axis=1)

In [18]:
%cd /Users/gardenmini/PycharmProjects/get_dataset/health

/Users/gardenmini/PycharmProjects/get_dataset/health


In [19]:
one_hot.rename(columns={'DaysInHospital': 'label'}, inplace=True)

In [24]:
one_hot.head()

,age_MISS,sexMALE,sexFEMALE,sexMISS,labNull,drugNull,no_Claims,no_Providers,no_Vendors,no_PCPs,...,drugCount_min,drugCount_ave,drugcount_months,labCount_max,labCount_min,labCount_ave,labcount_months,age,ClaimsTruncated,label
0,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2,0,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1


In [26]:
train, test = train_test_split(one_hot, test_size=0.2)
train.index = range(len(train))
test.index = range(len(test))
train.to_pickle('./health_train.pkl')
test.to_pickle('./health_test.pkl')